In [5]:
# !pip install PyKomoran
import pandas as pd
import re
from PyKomoran import *
import networkx as nx


In [6]:
# 텍스트 파일 불러오기
comment_df = pd.read_excel('여름의 너에게.xlsx')

# 불용어 리스트 만들기
f=open('stopwords.txt','r',encoding='utf-8')
stop_words = f.read().splitlines()


In [15]:
# Initialize Komoran tokenizer
komoran = Komoran("STABLE")

# Preprocessing function using KoNLPy2
def preprocess_text(text, stop_words):
    # Remove non-Korean characters and whitespace
    text = re.sub(r'[^가-힣\s]', '', text)
    # Tokenize the text using Komoran
    morphs_and_tags = komoran.get_plain_text(text).split()
    morphs = [mt.split('/')[0] for mt in morphs_and_tags]
    # Remove stopwords
    words = [word for word in morphs if word not in stop_words]
    return words

word_list = []
for text in comment_df['review']:
    words = preprocess_text(text,stop_words)
    word_list.append(words)
    


In [16]:
comment_df['preprocessed'] = word_list
print(comment_df['preprocessed'])

0                                             [시골, 너무, 좋]
1                                         [아따, 말, 잘생기, 엇]
2               [잘생기, ㄴ, 사람, 은, 그림, 체, 뽀, 요, 로맨스, 시작, ㄴ가]
3                                                [공포물, 았]
4                               [그림, 체, 말, 니, 바, 끼, 엇, ㅇ]
                              ...                        
4151    [작가, 님, 일러스트, 사랑, 합, 미, 다, 열심히, 정, 주행, 고, 기다리,...
4152                       [머리, 기른, 태민이, 미쳤다, 넘, 잘생겼어요, ]
4153                      [고등학생, 되, ㄴ, 애기, 시즌, 기대, 되, ㄴ다]
4154                                        [처음, 다, 보, 았]
4155                                            [댓, 글, 빠]
Name: preprocessed, Length: 4156, dtype: object


In [20]:
# 단어 그래프를 편집하는 함수
def word_graph(sentence):
    # Create a graph representation of the sentences
    
    for word in sentence:
        graph.add_node(word)
    for i in range(len(sentence)):
        for j in range(i+1, len(sentence)):
            word1, word2 = sentence[i], sentence[j]
            if graph.has_edge(word1, word2):
                graph[word1][word2]['weight'] += 1
            else:
                graph.add_edge(word1, word2, weight=1)

                
# 텍스트 랭킹 함수
def text_ranking(graph,num_keywords=50, damping_factor=0.85, max_iter=100, tol=1e-4):

    # PageRank algorithm to get keywords
    pagerank_scores = nx.pagerank(graph, alpha=damping_factor, max_iter=max_iter, tol=tol)

    # Sort the keywords based on their PageRank scores
    keywords = sorted(pagerank_scores, key=pagerank_scores.get, reverse=True)[:num_keywords]
    
    return keywords


# 그래프 생성
graph = nx.Graph()

for sentence in comment_df['preprocessed']:
    word_graph(sentence)

keywords = text_ranking(graph)


In [21]:
print(keywords)

['', '는', 'ㄴ', '다', '태민', '고', '너무', 'ㄹ', '았', '윤이', '소라', '되', '안', '주', '지', '도', '윤', '었', '있', '윤아', '은', '보', '게', '태', '좋', '귀엽', '진짜', '민이', '잘', '같', '남', '는데', 'ㄴ다', '그냥', '님', '작가', '면', '겠', 'ㅁ', '제발', '시', '말', '랑', '계속', '더', '내', '뺏기', '좋아하', '기', 'ㄴ데']


### 웹툰 특성 상 가장 많은 내용은 누구 예쁘다 잘생겼다, 제발 더 내주세요 등 분석에 필요없는 내용이 많음

### 추출한 키워드를 토대로 품사 단위로 구분, 이를 이용해서 문장 만들기


In [ ]:
def classify_keywords(keywords):
    

In [19]:
# Analysis on the keywords
def analyze_keywords(keywords_list):
    all_keywords = [keyword for keywords in keywords_list for keyword in keywords]
    keyword_freq = defaultdict(int)
    for keyword in all_keywords:
        keyword_freq[keyword] += 1
    sorted_freq = sorted(keyword_freq.items(), key=lambda x: x[1], reverse=True)
    return sorted_freq

# Get overall keyword analysis
overall_keywords_freq = analyze_keywords(keywords)

# Example: Top 10 most frequent keywords
top_10_keywords = overall_keywords_freq[:10]
print("Top 10 most frequent keywords:")
for keyword, frequency in top_10_keywords:
    print(f"{keyword}: {frequency} occurrences")


KeyError: 'keywords'